### Messing with decision trees

Reference: https://developers.google.com/machine-learning/decision-forests/growing

In [8]:
import pandas as pd
import numpy as np

In [5]:
DATA = "data/titanic/"

In [20]:
train_raw = pd.read_csv(f'{DATA}train.csv')
train_raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [23]:
males = train_raw.Sex == "male"
males = train_raw[males]
females = train_raw.Sex != "male"
females = train_raw[females]
np.average(males.Survived)

0.18890814558058924

In [24]:
np.average(females.Survived)

0.7420382165605095

Alright, let's make a decision tree. Let's take the sex as the first feature we will look into. Breaking up the data in this scenario
is pretty trivial since sex can only take two values in this dataset. Let's look into the basic principles of the approach behind decision trees.

The idea behind decision trees is that we want to create a hierarchy of features within a dataset, that allow us to describe, and better predict,
an outcome given some set of features about a datapoint. Let's take the example of the Titanic dataset. I want to, given the data provided above, predict
whether a person will survive or die. Using just information about sex, by using a naive approach, I'm going to make a simple model:

```
if male => will die
else => will live
```

In this scenario, i'm just rounding the probability that a lady would die vs. a male. This model is crude, but, in the case of males, will be correct about 3/4 of the time. To make it more sophisticated, I can start nesting conditions. This nesting allows us to better capture the complexity of the data within our model. Nesting, in this case, could mean something like "is male and age is greater than 30". Conditions for now are always true or false, with the nesting resulting in a binary tree. The leafs of our tree are the probability of our dependent variable being the positive case given the precedeing conditions (i.e. probability that they will live).

An important part of decision trees is determining the order and structure of the hierarchy, since all we start out with are a list of features that we're interested in. A good measurement fo how good a feature is for predicting the dependent variable is by measuring the information gain that feature provides. How do we measure information gain? What does that even mean? Let's just start with the intuition that an increase in information results in an increase in heterogeneity. Information allows us to separate data. More information is equivalent to saying less entropy. The big idea is that entropy makes data hard to distinguish, information allows us to distinguish it. Like putting on a new pair of glasses.

### Measuring entropy

$$\begin{eqnarray}
R(X) & = & \frac{\lvert \{ x | x \in X \ \textrm{and} \ x = \mathrm{pos}  \} \rvert}{\lvert X \rvert} \\[12pt]
H(X) & = & - p \log p - (1 - p) \log (1-p) \ \textrm{with} \ p = R(X)\\[12pt]
\end{eqnarray}$$

$R(X)$ is the proportion of the data in X that takes on the positive condition in the dependent variable.
$H(X)$ is the Shannon Entropy.

In [33]:
# Let's define the positive condition as being alive :)
# Let's find the proportion of males that ended up alive, and let's do the same for females

r_male = np.average(males.Survived)
r_female = np.average(females.Survived)

# Let's then calculate the entropy from these proportions

def entropy(p):
    return -p * np.log(p) - (1 - p) * np.log(1 - p)

h_male = entropy(r_male)
h_female = entropy(r_female)

In [37]:
# Let's also calculate the baseline entropy of the data

r_base = np.average(train_raw.Survived)
h_base = entropy(r_base)

In [39]:
h_base, h_female, h_male

(0.6659119735267652, 0.5709141922481396, 0.4846358858279689)

## Measuring information gain

Just take the base entropy, and subtract the weighted average of the entropies from the two splits.

$$\begin{eqnarray}
IG(D,T,F) & = & H(D) - \frac {\lvert T\rvert} {\lvert D \rvert } H(T) - \frac {\lvert F \rvert} {\lvert D \rvert } H(F)
\end{eqnarray}$$

IG -> Information Gained

Where $D$ represents the base set of the dependent variable, not filtered based on a feature's split (e.g. male vs. female for the sex feature)

In [44]:
ig = h_base - (males.size/train_raw.size) * h_male - (females.size/train_raw.size) * h_female
ig

0.15087048925218174

### Dang!

Now we have a way to evaluate how good a split of the data is based on a feature! Big IG means more good.